# Analysis of Scheduling Conflicts Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

congress = input("Congress (3-digit number): ")

# Specify directory for this congress
directory = str(congress)

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

# if int(congress) > 115:
#     results = pd.read_csv(str("C:\\Users\\rorey\\OneDrive - Bipartisan Policy Center\\Congress\\Modernization\\Scheduling Conflicts\\NEW\\Results\\"+str(congress)+"\\results"+str(congress)+".csv"))
#     hearing_data = pd.read_csv(str("C:\\Users\\rorey\\OneDrive - Bipartisan Policy Center\\Congress\\Modernization\\Scheduling Conflicts\\NEW\\Results\\"+str(congress)+"\\hearings"+str(congress)+".csv"))
#     members = pd.read_csv(str("C:\\Users\\rorey\\OneDrive - Bipartisan Policy Center\\Congress\\Modernization\\Scheduling Conflicts\\NEW\\Results\\"+str(congress)+"\\assignments"+str(congress)+".csv"),index_col=0)
    
# else:
#     results = pd.read_csv(str("C:\\Users\\rorey\\OneDrive - Bipartisan Policy Center\\Congress\\Modernization\\Scheduling Conflicts\\NEW\\Results\\2020 run\\results"+str(congress)+".csv"))
#     hearing_data = pd.read_csv(str("C:\\Users\\rorey\\OneDrive - Bipartisan Policy Center\\Congress\\Modernization\\Scheduling Conflicts\\NEW\\Results\\2020 run\\hearings"+str(congress)+".csv"))
#     members = pd.read_csv(str("C:\\Users\\rorey\\OneDrive - Bipartisan Policy Center\\Congress\\Modernization\\Scheduling Conflicts\\NEW\\Results\\2020 run\\assignments"+str(congress)+".csv"),index_col=0)    

results = pd.read_csv(directory + "/results.csv")
hearing_data = pd.read_csv(directory + "/hearings.csv")
members = pd.read_csv(directory + "/assignments.csv", index_col=0)


## Quick pulling together of number of assigments per member

In [2]:
members
members["id"] = members.index
members_long = pd.melt(members,id_vars=["id"])

In [3]:
# members_long.drop(["variable"],axis=1,inplace=True)
# members_long.dropna(inplace=True)

assign_dist = members_long.groupby(["value"]).count()
# assign_dist["member"] = assign_dist.index
assign_dist

assign_dist.to_csv(directory + "/assignmentsbymember.csv", index=True)

# Frequency of Conflicts by Committee

In [4]:
#committees with most conflicts, subcommittees broken out
hearings_combined = pd.concat([results["Hearing 1 Code"], results["Hearing 2 Code"]])

#import file with committee names
replacement = pd.read_csv(str("https://raw.githubusercontent.com/rachelorey/Scheduling-Conflicts-in-Congress/master/replacement"+str(congress)+".csv"))

#committes with most conflicts, sub > general
comm_freq  = pd.DataFrame(pd.DataFrame([i[0:2] for i in hearings_combined]).value_counts())
comm_freq.reset_index(inplace=True)
comm_freq.columns = ["Short Code","Scheduling Conflicts"]

comm_freq["Code"] = [i+"00" for i in comm_freq["Short Code"]]
comm_freq = pd.merge(replacement,comm_freq, on = "Code",how="inner")
comm_freq.drop("Short Code",axis=1,inplace=True)
comm_freq.sort_values(by="Scheduling Conflicts",ascending=False,inplace=True)

membercount = list()
for code in comm_freq["Code"]:
    membercount = membercount + [members.transpose()[code].count()]
    
comm_freq["Committee Members (Count)"] = membercount

display(comm_freq)

comm_freq.to_csv(directory + "/comm_freq.csv", index=False)

,Committee,Code,Scheduling Conflicts,Committee Members (Count)
30,Committee on Financial Services,BA00,1161,55
6,Financial Services,BA00,1161,55
36,Committee on Oversight and Reform,GO00,1144,46
12,Oversight and Reform,GO00,1144,46
35,Committee on Natural Resources,II00,1065,48
11,Natural Resources,II00,1065,48
16,Transportation and Infrastructure,PW00,1056,69
41,Committee on Transportation and Infrastructure,PW00,1056,69
10,Judiciary,JU00,1000,44
34,Committee on the Judiciary,JU00,1000,44


# Conflicts per Hearing

In [5]:
#reformatting dates
from datetime import datetime
import pandas as pd 

#results df
try:
    results["Date"] = [datetime.strptime(string, '%A, %B %d, %Y') for string in results["Date"]]
except:
    pass
results ["Month"] = [d.month for d in results["Date"]]
results ["Year"] = [d.year for d in results["Date"]]
results["Quarter"] = [(m-1)//3 + 1 for m in results["Month"]]

#hearings df
try:
    hearing_data["Date"] = [datetime.strptime(string, '%A, %B %d, %Y') for string in hearing_data["Date"]]
except:
    pass
hearing_data ["Month"] = [d.month for d in hearing_data["Date"]]
hearing_data ["Year"] = [d.year for d in hearing_data["Date"]]
hearing_data["Quarter"] = [(m-1)//3 + 1 for m in hearing_data["Month"]]

#setting up dataframes
conflictsxhearings_quarter = pd.DataFrame(columns=["year","quarter","conflicts","hearings"])
conflictsxhearings_month = pd.DataFrame(columns=["year","month","conflicts","hearings"])


unique_years = hearing_data["Year"].unique()

#for each year in the congress
for year in unique_years:

    for q in range(1,5):
        try:
            conflicts = results[results["Year"]==year]["Quarter"].value_counts()[q]
            hearings = hearing_data[hearing_data["Year"]==year]["Quarter"].value_counts()[q]
            res = pd.DataFrame([[year,q,conflicts,hearings]],columns = ["year","quarter","conflicts","hearings"])
            conflictsxhearings_quarter = pd.concat([conflictsxhearings_quarter, res])
        except:
            continue

    for m in range(1,13):
        try:
            conflicts = results[results["Year"]==year]["Month"].value_counts()[m]
        except:
            conflicts = 0
        try:
            hearings = hearing_data[hearing_data["Year"]==year]["Month"].value_counts()[m]
        except:
            hearings = 0
        res = pd.DataFrame([[year,m,conflicts,hearings]],columns = ["year","month","conflicts","hearings"])
        conflictsxhearings_month = pd.concat([conflictsxhearings_month, res])

print("\nConflicts per hearing per quarter")
display(conflictsxhearings_quarter)
print("\nConflicts per hearing per month")
display(conflictsxhearings_month)

conflictsxhearings_quarter.to_csv(directory + "/conflicts_by_hearing_quarter.csv")
conflictsxhearings_month.to_csv(directory + "/conflicts_by_hearing_month.csv")



Conflicts per hearing per quarter


,year,quarter,conflicts,hearings
0,2021,1,561,253
0,2021,2,1521,444
0,2021,3,647,214
0,2021,4,662,205
0,2022,1,867,240
0,2022,2,1680,382
0,2022,3,912,187
0,2022,4,152,71



Conflicts per hearing per month


,year,month,conflicts,hearings
0,2021,1,0,3
0,2021,2,197,93
0,2021,3,364,157
0,2021,4,419,117
0,2021,5,511,175
0,2021,6,591,152
0,2021,7,477,135
0,2021,8,0,6
0,2021,9,170,73
0,2021,10,240,94


In [6]:
#delete row from hearings.csv in the directory if date in 2024
hearings = pd.read_csv(directory + "/hearings.csv")
hearings["Date"]

0          Tuesday, January 12, 2021
1          Tuesday, January 12, 2021
2        Wednesday, February 3, 2021
3         Thursday, January 28, 2021
4         Thursday, February 4, 2021
                    ...             
1991     Thursday, December 15, 2022
1992       Monday, December 19, 2022
1993      Tuesday, December 20, 2022
1994    Wednesday, December 21, 2022
1995     Thursday, December 22, 2022
Name: Date, Length: 1996, dtype: object

# Conflicts per Individual Member, Hearing

## Members sorted by those with most conflicts

In [7]:
results["MC"].value_counts()

MC
Steve Cohen         73
Brad Sherman        71
Rashida Tlaib       71
Sylvia R. Garcia    69
Angie Craig         67
                    ..
Ted Budd             1
Jason Smith          1
Peter A. DeFazio     1
Tom Emmer            1
Ken Calvert          1
Name: count, Length: 368, dtype: int64

## Hearings with most conflicts

In [8]:
conflicts_hearing = pd.DataFrame(results["Hearing 1 Link"].append(results["Hearing 2 Link"]).value_counts(),columns=["Scheduling Conflicts"])
conflicts_hearing["Committee"] = [hearing_data[hearing_data["Link"]==link]["Committee"].values[0] for link in conflicts_hearing.index]
conflicts_hearing["Code"] = [hearing_data[hearing_data["Link"]==link]["Code"].values[0] for link in conflicts_hearing.index]
conflicts_hearing.index.rename("Link",inplace=True)

maxhearing = str(conflicts_hearing.reset_index().iloc[0,0])
maxcommittee = conflicts_hearing["Committee"].values[0]
maxcommittee_code = conflicts_hearing["Code"].values[0]
maxhearing_conflicts = results[(results["Hearing 1 Link"]==maxhearing)|(results["Hearing 2 Link"]==maxhearing)]
memberswconflict = len(maxhearing_conflicts.drop_duplicates(subset="MC"))
membersincomm = len(members.loc[str(maxcommittee_code)].dropna())


print("The",maxcommittee,"had the hearing with the single highest number of scheduling conflicts.\n\nLink to hearing:",maxhearing,end="\n\n")
print(memberswconflict,"members at the hearing had a conflict, out of",membersincomm,"total members.",end="\n\n")
print("That's",round(memberswconflict/membersincomm,2),"percent of membership with a conflict")

conflicts_hearing.plot.hist(title="Distribution of Conflicts per Committee Hearing",ylabel="test")
conflicts_hearing.to_csv(directory + "/conflictsperhearing.csv", index=True)

AttributeError: 'Series' object has no attribute 'append'

In [ ]:
print(round(len(conflicts_hearing)/len(hearing_data),2),"percent of hearings had at least one conflict")

In [ ]:
## GET AVERAGE NUMBER OF MEMBERS WITH CONFLICTS PER HEARING

membership = [len(members.loc[str(com)].dropna()) for com in members.index]
membership = pd.DataFrame(membership,index=members.index)

conflicts_hearing.reset_index(inplace=True)
averageconflicts = conflicts_hearing
averageconflicts.index = conflicts_hearing["Code"]
averageconflicts.drop(["Code"],axis=1,inplace=True)
averageconflicts = averageconflicts.join(membership,how='left')

averageconflicts = averageconflicts.rename(columns={0:"NumMembers"})

averageconflicts["Percent_Conflict"] = averageconflicts["Scheduling Conflicts"]/averageconflicts["NumMembers"]

print("Average Members in Each Hearing with a Conflict [excluding hearings with no conflicts]")
averageconflicts["Percent_Conflict"].mean()

In [ ]:
## MERGING WITH HEARING DATA TO GET PERCENT INCLUDING HEARINGS WITHOUT CONFLICTS

averageconflicts.reset_index(inplace=True)
averageconflicts.set_index("Link",inplace=True)
averageconflicts.drop(["Committee"],inplace=True,axis=1)

hearing_data.set_index("Link",inplace=True)

averageconflicts1 = hearing_data.join(averageconflicts,how="left")

averageconflicts1["Scheduling Conflicts"].fillna(0,inplace=True)
averageconflicts1["Percent_Conflict"].fillna(0,inplace=True)

print("Including hearings with no conflicts, average percent of members with a conflict overall")
print(averageconflicts1["Percent_Conflict"].mean())

# Conflicts by Committee Assignments

In [ ]:
import pandas as pd

member_count = pd.melt(members).drop(["variable"],axis=1)
member_count = member_count[~member_count["value"].isna()]
member_count = pd.DataFrame(pd.value_counts(member_count["value"]))
member_count.columns = ["No. Committees"]

conflict_count = pd.DataFrame(pd.value_counts(results["MC"]))
conflict_by_assignments = pd.merge(member_count,conflict_count,left_index = True,right_index=True)
conflict_by_assignments.columns = ["No. Committees","No. Conflicts"]

conflict_by_assignments.to_csv(directory + "/conflict_by_assignment.csv")
conflict_by_assignments.plot.scatter(x="No. Committees", y="No. Conflicts", title="Relationship between Committee Assignments and Scheduling Conflicts\n(including subcommittee assignments)")

In [ ]:
member_count["No. Committees"].mean()

In [ ]:
# hearing_data[hearing_data["Hearing Title"].str.contains("Mark up",case=False)|hearing_data["Hearing Title"].str.contains("Markup",case=False)]

# Overlap

In [ ]:
members.reset_index(inplace=True)
members.drop(["id"],axis=1,inplace=True)
members_long = members[members['index'].str.contains('00')]
members_long = pd.melt(members_long,id_vars=["index"])
members_long.drop(["variable"],axis=1,inplace=True)
members_long.dropna(inplace=True)

In [ ]:
hearing_data["ShortCode"] = hearing_data["Code"].str[:-2]
hearing_freq = hearing_data.groupby(["ShortCode"]).count()[["Date"]]
hearing_freq.to_csv(directory + "/hearingfreq.csv", index=True)

In [ ]:
comms = members_long["index"].unique()
overlap = pd.DataFrame(columns=comms,index=comms)

for comm in range(len(comms)):
    l1 = list(members_long[members_long['index']==comms[comm]]["value"])
    for comm2 in range(len(comms)):
        l2 = list(members_long[members_long['index']==comms[comm2]]["value"])
#         print(comm,comm2)
        numoverlap = len([value for value in l1 if value in l2])
        overlap.iloc[comm,comm2] = numoverlap

In [ ]:
overlap.to_csv(directory + "/overlap.csv",index=True)